# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 19 2022 12:11PM,242223,10,SONSB0001873,"Nextsource Biotechnology, LLC",Released
1,Jul 19 2022 12:10PM,242222,10,BPS1981,Beach Patient Assistance,Released
2,Jul 19 2022 12:09PM,242221,10,8278850,"ClearSpec, LLC",Released
3,Jul 19 2022 11:50AM,242219,19,ACG-2102490649,ACG North America LLC,Released
4,Jul 19 2022 11:49AM,242218,20,8294779,"Exact-Rx, Inc.",Released
5,Jul 19 2022 11:49AM,242218,20,8294780,"Exact-Rx, Inc.",Released
6,Jul 19 2022 11:49AM,242218,20,8295345,"Exact-Rx, Inc.",Released
7,Jul 19 2022 11:46AM,242217,18,6735952,Eye Pharma Inc,Released
8,Jul 19 2022 11:46AM,242213,10,MSP212295,"Methapharm, Inc.",Released
9,Jul 19 2022 11:46AM,242213,10,MSP212296,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
62,242218,Released,3
63,242219,Released,1
64,242221,Released,1
65,242222,Released,1
66,242223,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242218,NaN,NaN,3.0
242219,NaN,NaN,1.0
242221,NaN,NaN,1.0
242222,NaN,NaN,1.0
242223,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4.0,8.0,0.0
242008,0.0,0.0,1.0
242088,0.0,0.0,1.0
242090,0.0,0.0,1.0
242091,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241900,4,8,0
242008,0,0,1
242088,0,0,1
242090,0,0,1
242091,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,8,0
1,242008,0,0,1
2,242088,0,0,1
3,242090,0,0,1
4,242091,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241900,4,8,
1,242008,,,1
2,242088,,,1
3,242090,,,1
4,242091,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 19 2022 12:11PM,242223,10,"Nextsource Biotechnology, LLC"
1,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance
2,Jul 19 2022 12:09PM,242221,10,"ClearSpec, LLC"
3,Jul 19 2022 11:50AM,242219,19,ACG North America LLC
4,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc."
7,Jul 19 2022 11:46AM,242217,18,Eye Pharma Inc
8,Jul 19 2022 11:46AM,242213,10,"Methapharm, Inc."
19,Jul 19 2022 11:45AM,242215,10,"Methapharm, Inc."
24,Jul 19 2022 11:40AM,242209,20,"ACI Healthcare USA, Inc."
25,Jul 19 2022 11:39AM,242214,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 19 2022 12:11PM,242223,10,"Nextsource Biotechnology, LLC",,,1
1,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance,,,1
2,Jul 19 2022 12:09PM,242221,10,"ClearSpec, LLC",,,1
3,Jul 19 2022 11:50AM,242219,19,ACG North America LLC,,,1
4,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc.",,,3
5,Jul 19 2022 11:46AM,242217,18,Eye Pharma Inc,,,1
6,Jul 19 2022 11:46AM,242213,10,"Methapharm, Inc.",,,11
7,Jul 19 2022 11:45AM,242215,10,"Methapharm, Inc.",,,5
8,Jul 19 2022 11:40AM,242209,20,"ACI Healthcare USA, Inc.",,,1
9,Jul 19 2022 11:39AM,242214,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 12:11PM,242223,10,"Nextsource Biotechnology, LLC",1,,
1,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance,1,,
2,Jul 19 2022 12:09PM,242221,10,"ClearSpec, LLC",1,,
3,Jul 19 2022 11:50AM,242219,19,ACG North America LLC,1,,
4,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc.",3,,
5,Jul 19 2022 11:46AM,242217,18,Eye Pharma Inc,1,,
6,Jul 19 2022 11:46AM,242213,10,"Methapharm, Inc.",11,,
7,Jul 19 2022 11:45AM,242215,10,"Methapharm, Inc.",5,,
8,Jul 19 2022 11:40AM,242209,20,"ACI Healthcare USA, Inc.",1,,
9,Jul 19 2022 11:39AM,242214,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 12:11PM,242223,10,"Nextsource Biotechnology, LLC",1,,
1,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance,1,,
2,Jul 19 2022 12:09PM,242221,10,"ClearSpec, LLC",1,,
3,Jul 19 2022 11:50AM,242219,19,ACG North America LLC,1,,
4,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 12:11PM,242223,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance,1.0,NaN,NaN
2,Jul 19 2022 12:09PM,242221,10,"ClearSpec, LLC",1.0,NaN,NaN
3,Jul 19 2022 11:50AM,242219,19,ACG North America LLC,1.0,NaN,NaN
4,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc.",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 12:11PM,242223,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Jul 19 2022 12:10PM,242222,10,Beach Patient Assistance,1.0,0.0,0.0
2,Jul 19 2022 12:09PM,242221,10,"ClearSpec, LLC",1.0,0.0,0.0
3,Jul 19 2022 11:50AM,242219,19,ACG North America LLC,1.0,0.0,0.0
4,Jul 19 2022 11:49AM,242218,20,"Exact-Rx, Inc.",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4601821,74.0,58.0,3.0
102,242008,1.0,0.0,0.0
15,1210789,50.0,33.0,0.0
18,242217,1.0,0.0,0.0
19,1695286,7.0,16.0,18.0
20,1937058,128.0,26.0,4.0
21,2905410,12.0,0.0,0.0
22,484422,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4601821,74.0,58.0,3.0
1,102,242008,1.0,0.0,0.0
2,15,1210789,50.0,33.0,0.0
3,18,242217,1.0,0.0,0.0
4,19,1695286,7.0,16.0,18.0
5,20,1937058,128.0,26.0,4.0
6,21,2905410,12.0,0.0,0.0
7,22,484422,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,74.0,58.0,3.0
1,102,1.0,0.0,0.0
2,15,50.0,33.0,0.0
3,18,1.0,0.0,0.0
4,19,7.0,16.0,18.0
5,20,128.0,26.0,4.0
6,21,12.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,74.0
1,102,Released,1.0
2,15,Released,50.0
3,18,Released,1.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,18,19,20,21,22
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,18.0,4.0,0.0,0.0
Executing,58.0,0.0,33.0,0.0,16.0,26.0,0.0,0.0
Released,74.0,1.0,50.0,1.0,7.0,128.0,12.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,18,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,18.0,4.0,0.0,0.0
1,Executing,58.0,0.0,33.0,0.0,16.0,26.0,0.0,0.0
2,Released,74.0,1.0,50.0,1.0,7.0,128.0,12.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,18,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,18.0,4.0,0.0,0.0
1,Executing,58.0,0.0,33.0,0.0,16.0,26.0,0.0,0.0
2,Released,74.0,1.0,50.0,1.0,7.0,128.0,12.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()